In [19]:
import sympy as sp

# Definir variables simbólicas
t = sp.symbols('t')
u = sp.Function('u')(t)
V = sp.Function('V')(t)
L = sp.symbols('L')
a_rell = 118.9010  # Amplitud de relleno
delta_rell = 1044.4751  # Desfase de relleno
r_rell = 440.4874  # Tasa de relleno
dec_prec = 0.006  # Decremento de precipitación

# Evaporación
a_ev = 124.1845  # Amplitud de evaporación
delta_ev = -81.9441  # Desfase de evaporación
r_ev = 308.8465  # Tasa de evaporación
inc_ev = 0.001 # Incremento de evaporación

# Función f(t): Parte no homogénea
evaporation = (r_ev + a_ev * (-sp.cos(2 * sp.pi * t + delta_ev))) * sp.exp(inc_ev * t)
inflow = (r_rell + a_rell * sp.sin(2 * sp.pi * t + delta_rell)) * sp.exp(-dec_prec * t)
condition = -u - evaporation + inflow

# Definir el Hamiltoniano
H = -u**2 - V**2 + L * condition

# Calcular derivadas del Hamiltoniano
dH_du = sp.diff(H, u)  # Derivada respecto a u
dH_dV = sp.diff(H, V)  # Derivada respecto a V
dH_dL = sp.diff(H, L)  # Derivada respecto a L (lambda)
# segunda derivada de u
dH_du2 = sp.diff(dH_du, u)

# Mostrar las derivadas
print("Derivada del Hamiltoniano respecto a u:")
sp.pprint(dH_du)

print("Segunda derivada del Hamiltoniano respecto a u:")
sp.pprint(dH_du2)

print("\nDerivada del Hamiltoniano respecto a V:")
sp.pprint(dH_dV)

print("\nDerivada del Hamiltoniano respecto a L:")
sp.pprint(dH_dL)


Derivada del Hamiltoniano respecto a u:
-L - 2⋅u(t)
Segunda derivada del Hamiltoniano respecto a u:
-2

Derivada del Hamiltoniano respecto a V:
-2⋅V(t)

Derivada del Hamiltoniano respecto a L:
                                              0.001⋅t                          ↪
- (308.8465 - 124.1845⋅cos(2⋅π⋅t - 81.9441))⋅ℯ        + (118.901⋅sin(2⋅π⋅t + 1 ↪

↪                        -0.006⋅t       
↪ 044.4751) + 440.4874)⋅ℯ         - u(t)


In [3]:
print("\nDerivada del Hamiltoniano respecto a V:")
dH_dV



Derivada del Hamiltoniano respecto a V:


-2*V(t)

In [4]:
print("\nDerivada del Hamiltoniano respecto a L:")
(dH_dL)



Derivada del Hamiltoniano respecto a L:


-(308.8465 - 124.1845*cos(2*pi*t - 81.9441))*exp(0.001*t) + (118.901*sin(2*pi*t + 1044.4751) + 440.4874)*exp(-0.006*t) - u(t)

In [5]:
print("\nDerivada del Hamiltoniano respecto a L:")
(dH_dL)



Derivada del Hamiltoniano respecto a L:


-(308.8465 - 124.1845*cos(2*pi*t - 81.9441))*exp(0.001*t) + (118.901*sin(2*pi*t + 1044.4751) + 440.4874)*exp(-0.006*t) - u(t)

In [6]:
# igualar -L' = dH_dV
L_prime = -dH_dV
print("\nL' =", L_prime)

# igualar V' = dH_dL
V_prime = dH_dL
print("\nV' =", V_prime)




L' = 2*V(t)

V' = -(308.8465 - 124.1845*cos(2*pi*t - 81.9441))*exp(0.001*t) + (118.901*sin(2*pi*t + 1044.4751) + 440.4874)*exp(-0.006*t) - u(t)


In [ ]:
from sympy import symbols, Function, Eq, dsolve, exp, sin, cos, pi, lambdify
from scipy.integrate import quad
# importar N para convertir a números
from sympy import N


# Definir las variables y funciones
t = symbols('t')  # Variable independiente
V = Function('V')(t)  # Función dependiente V(t)
L = Function('L')(t)  # Función dependiente L(t)
U = Function('U')(t)  # Función dependiente U(t)

a_rell = 118.9010  # Amplitud de relleno
delta_rell = 1044.4751  # Desfase de relleno
r_rell = 440.4874  # Tasa de relleno
dec_prec = 0.0060  # Decremento de precipitación

# Evaporación
a_ev = 124.1845  # Amplitud de evaporación
delta_ev = -81.9441  # Desfase de evaporación
r_ev = 308.8465  # Tasa de evaporación
inc_ev = 0.0010  # Incremento de evaporación

# Definir la función f(t) (parte no homogénea)
f_t = (r_ev + a_ev * (-cos(2 * pi * t + delta_ev))) * exp(inc_ev * t) \
    - (r_rell + a_rell * sin(2 * pi * t + delta_rell)) * exp(-dec_prec * t)

# Plantear las ecuaciones diferenciales
eq1 = Eq(V.diff(t, 1), L / 2 - f_t)  # Ecuación para V'
eq2 = Eq(L.diff(t, 1), 2 * V)  # Ecuación para L'

# Resolver el sistema de ecuaciones diferenciales con condiciones iniciales
solution = dsolve([eq1, eq2], ics={V.subs(t, 0): 343.88, V.subs(t, 10): 328})

# Extraer las soluciones de V(t) y L(t)
V_solution = (solution[0].rhs)  # Solución para V(t)
L_solution = (solution[1].rhs)  # Solución para L(t)

# Definir u en función de L
u_solution = L_solution / (-2)

# Convertir soluciones simbólicas a funciones numéricas
V_numeric = lambdify(t, V_solution, modules=["numpy"])
L_numeric = lambdify(t, L_solution, modules=["numpy"])
u_numeric = lambdify(t, u_solution, modules=["numpy"])

# Definir la función integranda numérica
def integrand_numeric(t):
    return -(u_numeric(t))**2 -V_numeric(t)**2

# Calcular la integral numérica con quad
integral_result_numeric, error = quad(integrand_numeric, 0, 10)

# Mostrar el resultado
print(f"Resultado numérico de la integral: {integral_result_numeric:.4f}")
print(f"Error estimado: {error:.4e}")


Resultado numérico de la integral: -399922.5062
Error estimado: 2.3227e-04


In [23]:
print("V:",N(V_solution))
print("L:",N(L_solution))
print("u:",N(u_solution))

V: 0.0150756956165211/exp(-1.0*t)**1.0 - 1.47706922684521*exp(-1.006*t)*sin(6.28318530717959*t + 1044.4751)/exp(-1.0*t)**1.0 - 9.22534757833094*exp(-1.006*t)*cos(6.28318530717959*t + 1044.4751)/exp(-1.0*t)**1.0 - 218.930119284294*exp(-1.006*t)/exp(-1.0*t)**1.0 + 9.63862728822856*exp(-0.999*t)*sin(6.28318530717959*t - 81.9441)/exp(-1.0*t)**1.0 - 1.53250114236446*exp(-0.999*t)*cos(6.28318530717959*t - 81.9441)/exp(-1.0*t)**1.0 + 154.577827827828*exp(-0.999*t)/exp(-1.0*t)**1.0 + 9.63767486455108*exp(0.00099999999999989*t)*sin(6.28318530717959*t - 81.9441) + 1.53541747820042*exp(0.00099999999999989*t)*cos(6.28318530717959*t - 81.9441) - 154.268981018981*exp(0.00099999999999989*t) + 347.856955110542*exp(-1.0*t) + 1.46031568710624*exp(-0.00600000000000001*t)*sin(6.28318530717959*t + 1044.4751) - 9.23081898296759*exp(-0.00600000000000001*t)*cos(6.28318530717959*t + 1044.4751) + 221.573138832998*exp(-0.00600000000000001*t)
L: 0.0301513912330423/exp(-1.0*t)**1.0 - 2.95413845369042*exp(-1.006*t)